In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
# file_path = "./lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"

# df = pd.read_csv(
#     file_path, sep='\t', header=None,
#     names=[
#         'user_id', 'timestamp', 'artist_id', 'artist_name', 'track_id', 'track_name'
#     ],
#     skiprows=[
#         2120260-1, 2446318-1, 11141081-1,
#         11152099-1, 11152402-1, 11882087-1,
#         12902539-1, 12935044-1, 17589539-1
#     ]
# )
# df["timestamp"] = pd.to_datetime(df.timestamp)
# df.sort_values(['user_id', 'timestamp'], ascending=True, inplace=True)
# print(f'Number of Records: {len(df):,}\nUnique Users: {df.user_id.nunique()}\nUnique Artist:{df.artist_id.nunique():,}')
# df.head(5)

In [3]:
# save_filepath = os.path.join("./lastfm-dataset-1K", 'lastfm-dataset-1k.snappy.parquet')

# df.to_parquet(save_filepath, compression='GZIP', index=False)

In [4]:
# save_filepath = os.path.join("./lastfm-dataset-1K", 'lastfm-dataset-1k.snappy.parquet')

# df = pd.read_parquet(save_filepath)
# print(f'Number of Records: {len(df):,}\nUnique Users: {df.user_id.nunique()}\nUnique Artist:{df.artist_id.nunique():,}')
# df.head(5)

In [5]:
# df = df.loc[:, ["user_id", "timstamp", "track_id"]]
# data = df[df.track_id == np.nan]

# data.to_csv("lastfm_1k_user_brief_no_nan_dataset.csv", index=False)

In [6]:
data = pd.read_csv("lastfm_1k_user_brief_no_nan_dataset.csv")

data["timestamp"] = pd.to_datetime(data.timestamp)

In [7]:
n_track = data.track_id.nunique()

n_chosen_track = 200000

chosen_track = data.track_id.unique()[np.random.choice(range(n_track), n_chosen_track, replace=False)]

filter_data = data[data.track_id.isin(chosen_track)]

In [8]:
filter_data = filter_data.reset_index(drop=True)

In [9]:
filter_data

,user_id,timestamp,track_id
0,user_000001,2006-08-13 15:17:35+00:00,4c7d5a2c-e1da-4fe3-96e3-c40a7a6f59e8
1,user_000001,2006-08-13 15:23:08+00:00,6cb57d5e-da03-4bcd-b7b5-b08023890b42
2,user_000001,2006-08-13 15:36:22+00:00,df8c41f0-1c02-4060-81ca-035ef3085663
3,user_000001,2006-08-13 16:00:07+00:00,1c061863-1d3e-4066-aa93-5c9ce0bf72f2
4,user_000001,2006-08-13 17:56:14+00:00,9b0d3374-7968-4844-9bac-8b9397eccd42
...,...,...,...
3492302,user_001000,2009-05-02 18:42:49+00:00,d314d6d3-1cc7-4976-b432-5affe383a4dd
3492303,user_001000,2009-05-02 18:54:58+00:00,c0aa6407-7075-4345-9267-f26edc566a33
3492304,user_001000,2009-05-02 19:01:02+00:00,d0b89e4c-1b2d-4a52-807c-838ed2748198
3492305,user_001000,2009-05-03 03:55:48+00:00,8e88b199-70b7-4bbc-a673-2dfd3234e1a5


In [10]:
n_inter = len(filter_data)

session_len = 20

seperator = np.array(range(n_inter)) // session_len

In [11]:
last = 0

time_interval = "05:0:00"

def process1(ee: pd.DataFrame):
    global last
    
    ee.insert(0, "sep", seperator[:len(ee)] + last + 1)
    last = seperator[len(ee) - 1] + last + 1
    
    return ee

def process2(ee: pd.DataFrame):
    ee.insert(0, "keep", len(ee) >= 5 and ee.timestamp.max() - ee.timestamp.min() < pd.Timedelta(time_interval))
    
    return ee

In [12]:
split_dataset = filter_data.groupby("user_id").apply(lambda x: process1(x))

In [13]:
kept_dataset = split_dataset.groupby("sep").apply(lambda x: process2(x))

In [14]:
final_dataset = kept_dataset[kept_dataset.keep]

final_dataset = final_dataset.reset_index(drop=True).drop(columns=["keep"])

In [15]:
final_dataset.track_id.nunique()

35265

In [21]:
final_dataset.user_id.nunique()

551

In [27]:
len(final_dataset)

178218

In [25]:
final_dataset = final_dataset.drop(columns=["user_id"]).rename(columns={"sep": "session_id", "track_id": "item_id"})

In [28]:
final_dataset.to_csv("lastfm_1k_user_35k_item_178k_inters.csv", index=False)